In [1]:
import cv2
import numpy as np

# 영상의 기하학적 변환

## 전단 변환

In [5]:
src = cv2.imread('./data/tekapo.bmp')

height = src.shape[0]
width = src.shape[1]

Mx = 0.3

M = np.array([[1, Mx, 0],
              [0, 1, 0]], dtype=np.float64)

dst = cv2.warpAffine(src, M, (int(width + Mx*height) , height)) # dsize=(0, 0) : 원본영상과 동일한 사이즈

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 크기 변환

In [9]:
src = cv2.imread('./data/tekapo.bmp')

height = src.shape[0]
width = src.shape[1]

Sx = 1.2
Sy = 1.2

M = np.array([[Sx, 0, 0],
              [0, Sy, 0]], dtype=np.float64)

dst = cv2.warpAffine(src, M, (int(width*Sx), int(height*Sy))) # dsize=(0, 0) : 원본영상과 동일한 사이즈

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [11]:
src = cv2.imread('./data/tekapo.bmp')

#dst = cv2.resize(src, (0, 0), fx=1.2, fy=1.2)
dst = cv2.resize(src, (1920, 1280))

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 회전 변환

In [15]:
src = cv2.imread('./data/tekapo.bmp')

height = src.shape[0]
width = src.shape[1]

ceter = int(width/2), int(height/2)
angle = 20 # 반시계방향 20도
scale = 1 

M = cv2.getRotationMatrix2D(ceter, angle, scale)

dst = cv2.warpAffine(src, M, (0, 0)) # dsize=(0, 0) : 원본영상과 동일한 사이즈

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [20]:
# 90도 단위로 회전할때 rotate 함수 사용
src = cv2.imread('./data/tekapo.bmp')

dst1 = cv2.rotate(src, cv2.ROTATE_90_CLOCKWISE)
dst2 = cv2.rotate(src, cv2.ROTATE_90_COUNTERCLOCKWISE)

cv2.imshow('src', src)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

## 대칭 변환

In [24]:
src = cv2.imread('./data/tekapo.bmp')

dst = cv2.flip(src, 1) # flip code : 1(좌우반전), 0(상하반전), -1(좌우, 상하 모두 반전)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 투시변환

In [27]:
src = cv2.imread('./data/tekapo.bmp')

height = src.shape[0]
width = src.shape[1]

src_pts = np.array([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]], np.float32)
dst_pts = np.array([[60, 100], [width-20, 80], [width-10, height-10], [5, height-20]], np.float32)

M = cv2.getPerspectiveTransform(src_pts, dst_pts)
dst = cv2.warpPerspective(src, M, (0, 0))

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [58]:
def on_mouse(event, x, y, flag, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        src_pt_list.append([x, y])
        cv2.circle(src, (x, y), 5, (0, 0, 255), 5)
        cv2.imshow('src', src)
        if len(src_pt_list) == 4 :
            src_pts = np.array(src_pt_list, dtype=np.float32)
            M = cv2.getPerspectiveTransform(src_pts, dst_pts)
            dst = cv2.warpPerspective(src, M, (width, height))
            cv2.imshow('dst', dst)

src_pt_list = []
width, height = 200, 300
dst_pts = np.array([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]], np.float32) # 마우스로 클릭한 순서대로 결과에도 매칭
            
src = cv2.imread('./data/card.bmp')

cv2.imshow('src', src)
cv2.setMouseCallback('src', on_mouse)

cv2.waitKey()
cv2.destroyAllWindows()

# 에지 검출과 응용

## 마스크 기반 에지 검출 - 소벨 마스크

In [59]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

Mx = np.array([[-1, 0, 1],
              [-2, 0, 2],
              [-1, 0, 1]], dtype=np.float32)
My = np.array([[-1, -2, -1],
              [0, 0, 0],
              [1, 2, 1]], dtype=np.float32)

dx = cv2.filter2D(src, -1, Mx, delta=128)
dy = cv2.filter2D(src, -1, My, delta=128)

cv2.imshow('src', src)
cv2.imshow('dx', dx)
cv2.imshow('dy', dy)
cv2.waitKey()
cv2.destroyAllWindows()


In [61]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

# 미분필터(소벨필터)를 따로 준비하지 않고, 원본 이미지와 마스크 연산까지 해주는 함수
dx = cv2.Sobel(src, cv2.CV_32FC1, 1, 0) # x축으로 미분
dy = cv2.Sobel(src, cv2.CV_32FC1, 0, 1) # y축으로 미분
# cv2.Sobel() 함수는 cv2.filter2D()로 마스크 연산까지는 동일한 형변환은 안되어 있고 포화연산도 안되어 있는 상태

fmag = cv2.magnitude(dx, dy) # [x축으로 미분 , y축으로의 미분 ]=> 그래디언트, magnitude 함수가 그래디언트의 크기를 구해줌
mag = np.clip(fmag, 0, 255).astype(np.uint8)

T = 160
ret, dst = cv2.threshold(mag, T, 255, cv2.THRESH_BINARY)

cv2.imshow('src', src)
cv2.imshow('mag', mag)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 캐니 에지 검출기

In [65]:
src = cv2.imread('./data/lenna.bmp', cv2.IMREAD_GRAYSCALE)

dst = cv2.Canny(src, 80, 160) # low:high (1:2 or 1:3 Canny recommended)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 허프 변환 직선 검출

In [68]:
import math
math.pi

3.141592653589793

In [83]:
src = cv2.imread('./data/building.jpg', cv2.IMREAD_GRAYSCALE)

edge = cv2.Canny(src, 100, 200)

rho = 1 # or 2 : 숫자가 작을수록 더 정밀하게 검출하지만 연산시간이 더 걸림
theta = math.pi/180 # 라디안 단위
threshold = 160 # 축적배열의 숫자가 높다는 것은 직선을 이루는 점들이 많다는 뜻.
                 # 얼마나 큰 값을 직선으로 판단할지는 threshold에 달려있음
minLineLength = 50 # 검출할 선분의 최소길이
maxLineGap = 5 # 직선으로 간주할 최대 에지 점 간격

lines = cv2.HoughLinesP(edge, rho, theta, threshold,
                       minLineLength = minLineLength,
                       maxLineGap = maxLineGap)

print(edge.shape)
dst = cv2.cvtColor(edge, cv2.COLOR_GRAY2BGR) # 직선을 그릴 도화지 (3 채널 도화지)
print(dst.shape)

if lines is not None:
    for i in range(len(lines)):
        #print(lines[i][0])
        line = lines[i][0]
        pt1 = line[0], line[1]
        pt2 = line[2], line[3]
        cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('edge', edge)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

(600, 868)
(600, 868, 3)
